# Capítulo 02 - Instalação e Configuração do DuckDB

Este notebook explora as diferentes formas de instalar, configurar e usar o DuckDB em Python. Vamos aprender sobre conexões, configurações avançadas, extensões e boas práticas.

## 📚 Tópicos Abordados:
1. Instalação do DuckDB
2. Queries Simples Diretas
3. Conexão em Memória (Padrão)
4. Conexão com Arquivo Persistente
5. Context Manager (with statement)
6. Configurações Básicas
7. Configurações Avançadas
8. Boas Práticas em Ambientes Multi-Thread
9. Queries Encadeadas
10. Instalação e Uso de Extensões
11. Permitir Extensões Não Assinadas

## 1. Instalação do DuckDB

O DuckDB pode ser instalado via pip. É rápido e leve!

In [ ]:
# Instalar DuckDB
%pip install duckdb -q
print("✓ DuckDB instalado com sucesso!")

## 2. Queries Simples Diretas

O DuckDB permite executar queries SQL diretas sem criar uma conexão explícita.

### 2.1 Query Simples com duckdb.sql()

In [ ]:
import duckdb

# Query simples para testar - retorna 42
resultado = duckdb.sql("SELECT 42 AS resposta")
resultado.show()

print("\n✓ Query executada com sucesso!")

### 2.2 Mais Exemplos de Queries Diretas

In [ ]:
# Query com cálculos
print("Query com cálculos:")
duckdb.sql("SELECT 10 * 5 AS multiplicacao, 100 / 4 AS divisao").show()

# Query com strings
print("\nQuery com strings:")
duckdb.sql("SELECT 'Hello, DuckDB!' AS mensagem").show()

# Query com data/hora
print("\nQuery com data/hora:")
duckdb.sql("SELECT current_timestamp AS agora").show()

## 3. Conexão em Memória (Padrão)

A forma mais comum de usar DuckDB é com uma conexão em memória.

### Vantagens:
- ⚡ Extremamente rápida
- 🔒 Isolada (não interfere com outros processos)
- 🧹 Limpa automaticamente ao fechar

### Desvantagens:
- 📉 Dados perdidos ao fechar a conexão
- 💾 Limitada pela memória RAM disponível

In [ ]:
import duckdb

# Conexão padrão (em memória) - equivalente a duckdb.connect(':memory:')
con = duckdb.connect()

# Executar query
resultado = con.sql("SELECT 42 AS x, 'Em Memória' AS tipo")
resultado.show()

print("\n✓ Conexão em memória criada!")
print(f"Tipo de conexão: {type(con)}")

### 3.1 Criar Tabela em Memória

In [ ]:
# Criar tabela temporária
con.sql("CREATE TABLE usuarios (id INTEGER, nome VARCHAR)")
con.sql("INSERT INTO usuarios VALUES (1, 'Alice'), (2, 'Bob'), (3, 'Charlie')")

# Consultar
print("Tabela 'usuarios' em memória:")
con.table("usuarios").show()

print("\n✓ Tabela criada e populada!")

## 4. Conexão com Arquivo Persistente

Para dados persistentes, use uma conexão com arquivo.

### Vantagens:
- 💾 Dados persistem entre execuções
- 📦 Fácil de compartilhar (arquivo único)
- 🔄 Pode ser versionado

### Quando Usar:
- Dados que precisam ser salvos
- Análises que levam tempo
- Compartilhamento de datasets

In [ ]:
import duckdb
import os

# Criar ou conectar a um arquivo de banco de dados
con_file = duckdb.connect("file.db")

# Dropar tabela se existir (para evitar conflitos)
con_file.sql("DROP TABLE IF EXISTS test")

# Criar tabela (se não existir)
con_file.sql("CREATE TABLE IF NOT EXISTS test (i INTEGER, descricao VARCHAR)")

# Inserir dados
con_file.sql("INSERT INTO test VALUES (42, 'Primeiro registro'), (100, 'Segundo registro')")

# Consultar
print("Dados na tabela 'test':")
con_file.table("test").show()

# Fechar conexão
con_file.close()

print("\n✓ Dados salvos em 'file.db'")
print(f"Arquivo existe: {os.path.exists('file.db')}")

### 4.1 Reconectar e Verificar Persistência

In [ ]:
# Reconectar ao mesmo arquivo
con_file2 = duckdb.connect("file.db")

print("Dados ainda estão lá após reconexão:")
con_file2.table("test").show()

# Limpar para próximos testes
con_file2.sql("DROP TABLE test")
con_file2.close()

print("\n✓ Persistência verificada!")

## 5. Context Manager (with statement)

A forma mais segura e pythônica de trabalhar com conexões.

### Vantagens:
- 🔒 Fecha automaticamente
- 🛡️ Garante cleanup mesmo com erros
- 📝 Código mais limpo e legível

In [ ]:
import duckdb

# Usar context manager
with duckdb.connect("file.db") as con:
    # Criar e popular tabela
    con.sql("CREATE TABLE IF NOT EXISTS produtos (id INTEGER, nome VARCHAR, preco DECIMAL)")
    con.sql("""
        INSERT INTO produtos VALUES 
        (1, 'Notebook', 3500.00),
        (2, 'Mouse', 50.00),
        (3, 'Teclado', 150.00)
    """)
    
    # Consultar
    print("Produtos cadastrados:")
    con.table("produtos").show()
    
# Conexão é fechada automaticamente aqui!
print("\n✓ Conexão fechada automaticamente!")

### 5.1 Context Manager com Tratamento de Erros

In [ ]:
try:
    with duckdb.connect("file.db") as con:
        # Query que funciona
        con.sql("SELECT * FROM produtos WHERE preco < 200").show()
        print("\n✓ Query executada com sucesso!")
except Exception as e:
    print(f"❌ Erro: {e}")
finally:
    print("\n✓ Conexão sempre fechada, mesmo com erro!")

## 6. Configurações Básicas

DuckDB permite configurar vários aspectos do comportamento.

### 6.1 Limitar Threads

In [ ]:
import duckdb

# Limitar a 1 thread (útil para debugging)
con_single = duckdb.connect(config={'threads': 1})

# Verificar configuração
threads_config = con_single.sql("SELECT current_setting('threads') AS threads").fetchall()
print(f"Número de threads configurado: {threads_config[0][0]}")

con_single.close()
print("\n✓ Conexão com 1 thread criada!")

### 6.2 Verificar Threads Padrão

In [ ]:
# Conexão padrão usa todas as CPUs disponíveis
con_default = duckdb.connect()

threads_default = con_default.sql("SELECT current_setting('threads') AS threads").fetchall()
print(f"Threads padrão (baseado nas CPUs): {threads_default[0][0]}")

con_default.close()

## 7. Configurações Avançadas

Múltiplas configurações podem ser definidas ao criar a conexão.

### Configurações Comuns:
- `threads`: Número de threads para processamento paralelo
- `max_memory`: Limite de memória (ex: '4GB', '512MB')
- `default_order`: Ordem padrão (ASC ou DESC)
- `temp_directory`: Diretório para arquivos temporários

In [ ]:
import duckdb

# Múltiplas configurações
con_advanced = duckdb.connect(config={
    'threads': 4,              # Usar 4 threads
    'max_memory': '4GB',       # Limitar a 4GB de RAM
    'default_order': 'DESC'    # Ordem decrescente por padrão
})

# Verificar configurações
print("Configurações aplicadas:")
print(f"Threads: {con_advanced.sql('SELECT current_setting(\'threads\')').fetchall()[0][0]}")
print(f"Max Memory: {con_advanced.sql('SELECT current_setting(\'max_memory\')').fetchall()[0][0]}")
print(f"Default Order: {con_advanced.sql('SELECT current_setting(\'default_order\')').fetchall()[0][0]}")

con_advanced.close()
print("\n✓ Configurações avançadas aplicadas!")

### 7.1 Testar Ordem Padrão

In [ ]:
# Criar conexão com ordem DESC padrão
con_desc = duckdb.connect(config={'default_order': 'DESC'})

# Criar tabela de teste
con_desc.sql("CREATE TABLE numeros (valor INTEGER)")
con_desc.sql("INSERT INTO numeros VALUES (1), (5), (3), (9), (2)")

# ORDER BY sem especificar ASC/DESC usa o padrão
print("Ordenação padrão (DESC configurado):")
con_desc.sql("SELECT * FROM numeros ORDER BY valor").show()

con_desc.close()

## 8. Boas Práticas em Ambientes Multi-Thread

**⚠️ IMPORTANTE**: Cada thread deve ter sua própria conexão!

### ✅ BOM: Cada thread com sua conexão

In [ ]:
import duckdb
import threading

# ✅ CORRETO: Cada thread cria sua própria conexão
def boa_pratica(thread_id):
    con = duckdb.connect()  # Nova conexão para esta thread
    resultado = con.sql(f"SELECT {thread_id} AS thread_id, 42 * {thread_id} AS calculo").fetchall()
    print(f"Thread {thread_id}: {resultado}")
    con.close()

# Criar e executar 3 threads
print("✅ Boa prática - cada thread com sua conexão:")
threads = []
for i in range(1, 4):
    t = threading.Thread(target=boa_pratica, args=(i,))
    threads.append(t)
    t.start()

# Aguardar todas as threads
for t in threads:
    t.join()

print("\n✓ Todas as threads completaram com sucesso!")

### ❌ EVITAR: Compartilhar conexão entre threads

**Não faça isso em produção!** Demonstração apenas educacional.

## 9. Queries Encadeadas

DuckDB permite referenciar resultados de queries anteriores.

### 9.1 Exemplo Básico

In [ ]:
import duckdb

# Primeira query armazena resultado em r1
r1 = duckdb.sql("SELECT 42 AS i, 'DuckDB' AS nome")

# Segunda query referencia r1
print("Resultado original (r1):")
r1.show()

print("\nQuery encadeada usando r1:")
duckdb.sql("SELECT i * 2 AS dobro, nome || ' é ótimo!' AS mensagem FROM r1").show()

print("\n✓ Queries encadeadas executadas!")

### 9.2 Múltiplas Queries Encadeadas

In [ ]:
# Query 1: Criar dataset
vendas = duckdb.sql("""
    SELECT * FROM (
        VALUES 
        ('Janeiro', 1000),
        ('Fevereiro', 1500),
        ('Março', 1200),
        ('Abril', 1800)
    ) AS t(mes, valor)
""")

print("Vendas originais:")
vendas.show()

# Query 2: Adicionar meta
com_meta = duckdb.sql("SELECT mes, valor, valor * 1.1 AS meta FROM vendas")

print("\nVendas com meta (+10%):")
com_meta.show()

# Query 3: Calcular status
print("\nVendas vs Meta:")
duckdb.sql("""
    SELECT 
        mes,
        valor,
        meta,
        CASE 
            WHEN valor >= meta THEN '✓ Atingiu'
            ELSE '✗ Não atingiu'
        END AS status
    FROM com_meta
""").show()

print("\n✓ Pipeline de queries encadeadas completo!")

## 10. Instalação e Uso de Extensões

DuckDB suporta extensões para funcionalidades adicionais.

### Repositórios Disponíveis:
- **core**: Extensões oficiais do DuckDB
- **community**: Extensões da comunidade

### 10.1 Listar Extensões Disponíveis

In [ ]:
import duckdb

con = duckdb.connect()

# Listar extensões disponíveis
print("Extensões Disponíveis (primeiras 10):")
con.sql("""
    SELECT extension_name, installed, loaded, description
    FROM duckdb_extensions()
    LIMIT 10
""").show()

con.close()

### 10.2 Instalar Extensão da Comunidade

Exemplo com extensão H3 (geoespacial)

In [ ]:
import duckdb

con = duckdb.connect()

try:
    # Instalar extensão H3 do repositório community
    print("Instalando extensão H3...")
    con.install_extension("h3", repository="community")
    
    # Carregar extensão
    print("Carregando extensão H3...")
    con.load_extension("h3")
    
    print("\n✓ Extensão H3 instalada e carregada!")
    
    # Testar extensão (se instalou com sucesso)
    print("\nTestando função H3:")
    con.sql("SELECT h3_cell_to_lat_lng('85283473fffffff') AS coordenadas").show()
    
except Exception as e:
    print(f"Nota: {e}")
    print("(Extensões podem requerer permissões adicionais ou configuração)")

con.close()

### 10.3 Extensões Core Comuns

In [ ]:
con = duckdb.connect()

# Exemplos de extensões core úteis
extensoes_uteis = [
    ('httpfs', 'Ler arquivos via HTTP/HTTPS/S3'),
    ('json', 'Trabalhar com JSON'),
    ('parquet', 'Ler/escrever Parquet'),
    ('postgres_scanner', 'Consultar PostgreSQL'),
    ('sqlite_scanner', 'Consultar SQLite'),
    ('excel', 'Ler arquivos Excel')
]

print("Extensões Core Úteis:")
print("=" * 60)
for nome, desc in extensoes_uteis:
    print(f"  {nome:20} - {desc}")

con.close()
print("\n✓ Use install_extension('nome') para instalar!")

## 11. Permitir Extensões Não Assinadas

⚠️ **ATENÇÃO**: Use apenas em ambientes de desenvolvimento!

Para extensões não assinadas ou de terceiros.

In [ ]:
import duckdb

# Permitir extensões não assinadas
con_unsigned = duckdb.connect(config={"allow_unsigned_extensions": "true"})

print("⚠️  Extensões não assinadas permitidas!")
print("")
print("Importante:")
print("  ✓ Use apenas em desenvolvimento")
print("  ✗ NÃO use em produção")
print("  ✓ Verifique a origem das extensões")
print("  ✗ Cuidado com segurança")

# Verificar configuração
config = con_unsigned.sql("SELECT current_setting('allow_unsigned_extensions') AS config").fetchall()
print(f"\nConfigurações: allow_unsigned_extensions = {config[0][0]}")

con_unsigned.close()

## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **Instalação** simples via pip
2. ✅ **Queries diretas** com `duckdb.sql()`
3. ✅ **Conexões em memória** (rápidas e temporárias)
4. ✅ **Conexões persistentes** com arquivos
5. ✅ **Context managers** (boas práticas)
6. ✅ **Configurações básicas** (threads, memória)
7. ✅ **Configurações avançadas** (múltiplas opções)
8. ✅ **Multi-threading** (uma conexão por thread)
9. ✅ **Queries encadeadas** (reuso de resultados)
10. ✅ **Extensões** (funcionalidades extras)
11. ✅ **Segurança** (extensões não assinadas)

### 🔑 Pontos-Chave:
- DuckDB é **leve e rápido**
- Suporta **memória e arquivo**
- **Altamente configurável**
- **Rico ecossistema** de extensões
- **Thread-safe** com uso correto

### 📚 Próximo Capítulo:
Importação e Exportação de CSV!

## 🧹 Limpeza (Opcional)

In [ ]:
import os

# Remover arquivo de teste
if os.path.exists("file.db"):
    os.remove("file.db")
    print("✓ Arquivo 'file.db' removido")
else:
    print("Arquivo 'file.db' não encontrado")